In this Cell we drop 'url', 'urlid', 'IP' and 'region' and 'bidid' from the training data.

In [ ]:
import os
import csv
import numpy as np
import random
from random import shuffle
import math
import pandas as pd
from random import uniform
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split


filename = '../input/train.csv'

data = pd.read_csv(filename)
#train = pd.read_csv('train.csv')

train = data.drop(['bidid',  'region',  'bidprice','url','urlid','IP'], axis=1)
#data = data.drop(['bidid',  'region',  'bidprice', 'url','urlid'],axis=1)

train['slotvisibility'] = pd.to_numeric(train['slotvisibility'], errors='coerce')
avgSlot = train['slotvisibility'].mean()

vals = '../input/validation.csv'

valid = pd.read_csv(vals)
#train = pd.read_csv('train.csv')

valid = valid.drop(['bidid',  'region',  'bidprice','url','urlid','IP'], axis=1)
valid['slotvisibility'] = pd.to_numeric(valid['slotvisibility'], errors='coerce')



fn = '../input/test.csv'

tests = pd.read_csv(fn)

tests = tests.drop(['region','url','urlid','IP'], axis=1)
#Here we separate out clicks and payprice data from feature values

ids = tests['bidid']
tests = tests.drop(['bidid'], axis=1)
clicks = train['click']
train = train.drop(['click'], axis=1)
price = train['payprice']
train = train.drop(['payprice'], axis=1)



cVal = valid['click']
valid = valid.drop(['click'], axis=1)
vPrice = valid['payprice']
valid = valid.drop(['payprice'], axis=1)

We append validation and test data during feature engineering to ensure all data goes through the same tranformation

In [ ]:
train=train.append(valid, ignore_index=True)
train=train.append(tests, ignore_index=True)

here we iterate through all records and determine what usertags are present, recording for each tag the sample in which it occurs.

In [ ]:
from tqdm import tqdm
from sklearn import preprocessing
def getDict(train):
    
    tagD = {}

    for index, row in tqdm(train.iterrows()):
        for j in [x.strip() for x in row['usertag'].split(',')]:
            if j in tagD:
                tagD[j].append(index)
            else:
                tagD[j]=[index]
    
    
    return tagD




In [ ]:
train['usertag']=train['usertag'].fillna("")
tagD = getDict(train)

tagD.pop('', None)
    
    
for i in tagD:
    train[i]=0




We one-hot the usertags by accessing for each one the samples in which they occur

In [ ]:



for i in tqdm(tagD.keys()):
    
    
    train.iloc[tagD[i], train.columns.get_loc(i)] = 1

     

In [ ]:
train = train.drop(['usertag'], axis=1)

In [ ]:
from tqdm import tqdm
def transformDatas(df, avgSlot):
    
    df = df.replace("FourthView","4")
    df = df.replace("OtherView","0")
    df = df.replace("FirstView","1")
    df = df.replace("ThirdView","3")
    
    df['sin_time'] = np.sin(2*np.pi*df.hour/24)#encode circularity of 24 h clock
    df['cos_time'] = np.cos(2*np.pi*df.hour/24)
    df.drop('hour', axis=1, inplace=True)
    
    df['slotvisibility'] = pd.to_numeric(df['slotvisibility'], errors='coerce')
    df['slotvisibility'] = df['slotvisibility'].fillna(avgSlot)
    
   
    df = df.drop(['userid'],axis=1)
    
    df['keypage'].fillna("", inplace=True)
    
    df['domain'].fillna("", inplace=True)
    
    df['adexchange'].fillna("", inplace=True)
    
    
    
    return df
    
    

In [ ]:

train = transformDatas(train, avgSlot)


In [ ]:
for f in train.columns: 
    if train[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[f].values)) 
        train[f] = lbl.transform(list(train[f].values))

In [ ]:
tests = train.tail(len(tests))
train = train.head(len(train)-len(tests))
valid = train.tail(len(valid))
train = train.head(len(train)-len(valid))

In [ ]:
import lightgbm as lightgbm
#assign categorical data
cols = set(['useragent', 'adexchange', 'domain','IP', 'slotid','slotformat','creative','keypage'])


Y = clicks
X = train


x, x_test, y, y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)



categorical_features = [c for c, col in enumerate(train.columns) if col in cols]
train_data = lightgbm.Dataset(x, label=y, categorical_feature=categorical_features)
test_data = lightgbm.Dataset(x_test, label=y_test)




parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    'boosting': 'gbdt',   #gbdt
    'num_leaves': 300,  #31
    'feature_fraction': 0.8,  #0.5
    'bagging_fraction': 0.6, #0.5
    'bagging_freq': 70, #20
    'learning_rate': 0.00001,
    'verbose': 0,
    'min_data_in_leaf':1000,
    'max_bin':1000,
    
    
}

model = lightgbm.train(parameters,
                       train_data,
                       valid_sets=test_data,num_boost_round=5000,early_stopping_rounds=100
                       );

In [ ]:
import matplotlib.pyplot as plt
print('Plotting feature importances...')
ax = lightgbm.plot_importance(model, max_num_features=20)
plt.show()

In [ ]:
avgCtr = 1793/(2429188+1793)

In [ ]:


true = pd.Series(clicks).values
price = pd.Series(price).values

In [ ]:
vTrue = pd.Series(cVal).values
vPrice = pd.Series(vPrice).values

In [ ]:
v_pred = model.predict(valid, num_iteration=model.best_iteration)

In [ ]:
roc_auc_score(vTrue, v_pred)

In [ ]:
clist = []
highest_pay_price = max(price)+1

def clicksAtPrice(price, bid, y_pred, true):
  clicks = 0
  budget = 6250
  for i, p in enumerate(price):

    if budget >= p/1000 and bid*(y_pred[i]/avgCtr) >= p:
      budget -= p/1000
      clicks += true[i]

  return clicks

mClick = 0
bid = 0
for i in tqdm(range(0, 60)):
    
    a = clicksAtPrice(vPrice,i,v_pred, vTrue)
    clist.append(a)
    
    if a>mClick:
        mClick=a
        bid = i
        
  

In [ ]:
bid_pred = model.predict(tests, num_iteration=model.best_iteration)
biid = pd.Series(ids).values

return linear bid strategy for test data

In [ ]:
import csv

with open('bids.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['bidid']  + ['bidprice']) 
    for i in range(0,len(biid)):
        writer.writerow([biid[i]]+ [13*(bid_pred[i]/avgCtr)])